In [42]:
import pandas as pd
from openai import OpenAI
import requests
import os
import base64
import re
from dotenv import load_dotenv # env 가져오기

#######################################
## Loaded env (for API_Key)
load_dotenv() 
## Loaded API KEY
api_key = os.getenv("API_KEY")

## Data download from github and save to PATH
# Path 지정
PATH = "[FILE_PATH]"
os.makedirs(PATH, exist_ok=True)

## Setting OpenAI API
# pip install "openai==1.66.0"
client = OpenAI(api_key=api_key)

## Check Clinical Note Examples

**1) Clinical Note 불러 오는 방법 안내**

파일에서 Clinical Note 불러오기

- 텍스트(.txt) 파일에 작성된 노트를 불러오는 방식입니다.
- 불러올 파일 경로를 정확히 지정해야 합니다.


**2) 예시로 사용되는 note1과 note2는 서로 다른 내용입니다:**
- note1: shorthand 스타일의 간단한 요약형 노트
- note2: narrative 스타일의 서술형 노트로, 보다 상세한 문맥과 진단 경과가 포함되어 있습니다

이 두 노트를 각각 사용하여 prompt 차이, 언어 차이, 노트 형식 차이에 따른 분석 결과 차이를 실험해볼 수 있습니다.

### 1) Shorthand version

In [2]:
## (1) Load note from .txt file
note1_path = os.path.join(PATH, "note1_ShorthandStyle.txt")
with open(note1_path, "r", encoding="utf-8") as f:
    note1 = f.read()

In [3]:
## check loaded note
print(note1)

#. T2DM  
   Dx since 2010, recent HbA1c 9.1  
   MTF → 설사, 복부팽만, 식욕저하, 무기력감  
   Lactic acidosis (Lac 3.9) 의심 → 약 중단, 수액 치료  
   MTF-induced lactic acidosis 의심 → 약 중단 후 회복, basal insulin 예정

   Levemir 22 --- 10 units, 저녁 인슐린 잘 안 맞음


#. HTN  
   BP on admission 92/58
   Losartan 유지 

#. CKD stage 3  
   Baseline Cr 1.6 → 입원 시 Cr 2.1  
   수액 이후 신기능 호전, 투석 필요 없음  

#. Dyslipidemia  
   Atorvastatin  


### 2) Narrative version (MIMIC-style)

In [4]:
## (1) Load note from .txt file
with open(f"{PATH}note2_MIMICstyle.txt", "r", encoding="utf-8") as f:
    note2 = f.read()

In [5]:
## check loaded note
print(note2[:500])  # 앞 500자만 출력

History of Present Illness:  
78-year-old female with a history of type 2 diabetes, hypertension, and CKD stage 3 who presented with generalized weakness, poor oral intake, and nausea.

Two weeks prior to admission, she was started on metformin after a primary care visit revealed HbA1c of 9.1. Since initiation, the patient has developed persistent loose stools and intermittent abdominal cramping. No reported fever or vomiting. Her family noted a gradual decline in appetite and energy over the pa


**[Important]**

본 실습에서는 **note1 (shorthand version)** 을 사용합니다.  
Narrative version을 사용하려면 아래 코드를 수정하세요.

In [6]:
# mimic version인 note2 사용
# note = note2

# shorthand version인 note1 사용
note = note1

# 1. Clinical Note Analysis

## [Task 1] Drug Name Extraction and drug code identification

이 Task에서는 임상노트로부터 언급된 약물명을 추출하는 작업을 수행합니다.

- GPT에게 임상노트를 입력하고, 텍스트에서 **약물명** 추출 및 약물에 상응하는 **ATC CODE**를 추출하게 합니다.
- 약물은 **일반명** 또는 **약어**(예: MFM = metformin)로 표현될 수 있습니다.



### (1) Drug Entity Recognition

 Zero-shot prompting 예시

In [16]:
prompt_drug_name = f"""

Task: Extract all drug names mentioned in the following clinical note.
- Drug names may be written as abbreviations.
- Return the list of drug names only, without any additional explanation.

Clinical Note:
{note}
"""

In [17]:
model="gpt-5" # select model

response = client.responses.create(
    model=model,
    instructions="You are a clinical text extraction assistant.", # system role
    input=prompt_drug_name
)

print(response.output_text)

MTF
insulin
Levemir
Losartan
Atorvastatin


### (2) Drug Entity + Description + ATC Code

few-shots prompting 예시

In [18]:
prompt_wATC = f"""

## Task
Given a clinical note, extract:
1. Drug names clearly or implicitly mentioned (including abbreviations).
2. A short description of each drug (1 line).
3. The ATC code corresponding to the drug (ATC Level 5).

Output must be in CSV format with the columns:
Drug, Description, ATC_Code

## Examples:
Clinical Note 1:
"ASA 투여 후 환자 두드러기 호소. PRN으로 항히스타민제 처방."

Output (CSV):
Drug,Description,ATC_Code
ASA,Aspirin (NSAID used for pain/fever, antiplatelet),B01AC06
항히스타민제,Antihistamine for allergic symptoms,R06AE07

Clinical Note 2:
"ASA 투여 후 환자 두드러기 호소. PRN으로 항히스타민제 처방."

Output (CSV):
Drug,Description,ATC_Code
ASA,Aspirin (NSAID used for pain/fever, antiplatelet),B01AC06
항히스타민제,Antihistamine for allergic symptoms,R06AE07

Now process the user input below.
Clinical Note:
{note}
"""

In [19]:
response = client.responses.create(
    model=model,
    instructions="You are a clinical text extraction assistant specializing in drug detection from clinical notes.", # system role
    input=prompt_wATC
)

print(response.output_text)

Drug,Description,ATC_Code
MTF,Metformin (biguanide for T2DM; GI upset; lactic acidosis risk),A10BA02
Levemir,Insulin detemir (long-acting basal insulin for diabetes),A10AE05
Losartan,ARB antihypertensive (HTN; renal protection in CKD),C09CA01
Atorvastatin,Statin (HMG-CoA reductase inhibitor) for dyslipidemia; lowers LDL,C10AA05


### (3) Usage check
사용한 토큰을 아래의 코드를 이용해서 체크할 수 있습니다.

In [20]:
print(response.usage)

ResponseUsage(input_tokens=460, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=1972, output_tokens_details=OutputTokensDetails(reasoning_tokens=1856), total_tokens=2432)


## [Task 2] Adverse Drug Reaction Detection

이 Task에서는 임상노트에 언급된 **약물 부작용(Adverse Drug Reactions, ADR)** 을 탐지하는 작업을 수행합니다.

- GPT에게 임상노트를 입력하고, 특정 약물 투여 이후 발생한 부작용을 **약물-증상 쌍(drug → symptom)** 형태로 추출하게 합니다.
- 약물명은 약어로 표현될 수 있으며, 부작용은 명시적 또는 암시적으로 표현되어 있을 수 있습니다.




### (1) ADR Agent

In [21]:
## Prompt
prompt_adr = f"""

Task: Extract all possible adverse drug reactions (ADRs) mentioned in the clinical note.
- Return results in JSON format, with each item containing:
  - "drug": drug name
  - "symptom": symptom or adverse effect
  - "evidence": supporting sentence or phrase from the note
- No extra explanation.

Clinical Note:
{note}
"""

In [23]:

response = client.responses.create(
    model=model,
    instructions="You are a clinical text extraction assistant specializing in identifying adverse drug reactions (ADRs).",
    input=prompt_adr
)

print(response.output_text)

[
  {
    "drug": "Metformin (MTF)",
    "symptom": "diarrhea",
    "evidence": "MTF → 설사, 복부팽만, 식욕저하, 무기력감"
  },
  {
    "drug": "Metformin (MTF)",
    "symptom": "abdominal bloating",
    "evidence": "MTF → 설사, 복부팽만, 식욕저하, 무기력감"
  },
  {
    "drug": "Metformin (MTF)",
    "symptom": "decreased appetite",
    "evidence": "MTF → 설사, 복부팽만, 식욕저하, 무기력감"
  },
  {
    "drug": "Metformin (MTF)",
    "symptom": "lethargy",
    "evidence": "MTF → 설사, 복부팽만, 식욕저하, 무기력감"
  },
  {
    "drug": "Metformin (MTF)",
    "symptom": "lactic acidosis",
    "evidence": "MTF-induced lactic acidosis 의심 → 약 중단 후 회복, basal insulin 예정"
  }
]


### (2) Verification Agent

위의 LLM이 생성한 답변이 맞는지 검증하기 위한 Agent를 생성해서 답변을 검증합니다.

In [25]:
verifier_prompt = """
If you disagree, provide counterarguments.
Provide agree and comment for each drug and side effect pairs.
Respond in JSON:
{"agree": true/false, "comments": "..."}.

"""

extractor_output = response.output_text
verifier_input=f"{verifier_prompt}\n\nExtractorAgent output:\n{extractor_output}\n\nMedical note:\n{note}"


In [31]:
model="gpt-5-mini" # change model

verifier_response = client.responses.create(
    model=model,
    instructions="You are VerifierAgent. Your job is to check ExtractorAgent's identified side effects.",
    input=verifier_input
)

print(verifier_response.output_text)


[
  {
    "drug": "Metformin (MTF)",
    "symptom": "diarrhea",
    "agree": true,
    "comments": "Agree. The clinical note explicitly lists '설사' and diarrhea is a well‑known, common gastrointestinal adverse effect of metformin."
  },
  {
    "drug": "Metformin (MTF)",
    "symptom": "abdominal bloating",
    "agree": true,
    "comments": "Agree. The note documents '복부팽만'. Bloating and other GI complaints are commonly reported with metformin."
  },
  {
    "drug": "Metformin (MTF)",
    "symptom": "decreased appetite",
    "agree": true,
    "comments": "Agree. The note includes '식욕저하'. Metformin can cause anorexia/reduced appetite as part of its GI side‑effect profile."
  },
  {
    "drug": "Metformin (MTF)",
    "symptom": "lethargy",
    "agree": true,
    "comments": "Agree, with caveat. The note lists '무기력감'. Lethargy can be reported with metformin but is nonspecific and may reflect underlying illness or metabolic disturbance (e.g., evolving lactic acidosis, volume depletion, ur

## [Task 3] Clinical Note Summarization

이 Task에서는 임상노트에 포함된 핵심 정보를 GPT를 통해 한글로 **요약(summarization)** 하는 작업을 수행합니다.
- 임상노트에는 환자의 병력, 진단, 투약, 검사 결과, 치료 계획 등의 정보가 포함되어 있으며, 종종 매우 길고 복잡합니다.
- 본 Task의 목적은 다음 두 가지 유형의 요약을 생성하는 것입니다:
    - 환자/보호자용 요약
    - 의료진용 요약
- 출력은 한국어 요약문으로 생성됩니다.



In [26]:
prompt_family = f"""
Task: Summarize the following clinical note in simple language for the patient’s family.
- Use non-technical, plain Korean language.
- The summary should explain what happened, what was diagnosed, how it was treated, and what to expect.
- Avoid complex terminology and abbreviations.

Output language: Korean

Clinical Note:
{note}
"""

In [27]:
## Family-Friendly Summary
model = "gpt-5" # select model

response = client.responses.create(
    model=model,
    instructions="You are a clinical text summarization assistant.",
    input=prompt_family
)

print(response.output_text)

가족분들을 위한 요약

무엇이 있었는지
- 당뇨병으로 약을 드시던 중, 설사·배가 더부룩함·식욕저하·심한 피로가 생겨 입원하셨습니다.
- 오실 때 혈압이 낮았고, 콩팥(신장) 기능 수치도 평소보다 나빠져 있었습니다.

무슨 문제로 보였는지(진단)
- 당뇨병 조절이 최근 잘 되지 않았습니다(장기 혈당 지표가 높게 나왔습니다).
- 당뇨약(메트포르민 성분) 때문에 몸에 산성 물질이 쌓이는 부작용이 생긴 것으로 의심되었습니다.
- 콩팥 기능은 원래 약한 편인데, 입원 시 일시적으로 더 나빠졌던 상태였습니다.
- 혈압이 낮게 측정되었고, 콜레스테롤(혈중 지방) 문제로 약을 복용 중이십니다.

어떻게 치료했는지
- 문제를 일으킨 것으로 의심되는 당뇨약(메트포르민)을 즉시 중단했습니다.
- 수액(정맥 주사로 수분 보충) 치료를 했고, 몸 상태와 콩팥 수치가 좋아졌습니다. 투석은 필요하지 않았습니다.
- 앞으로는 당뇨 조절을 위해 인슐린 주사로 바꾸어 치료할 계획입니다. 저녁 주사를 자주 놓치셔서, 하루 한 번 맞는(오래 가는) 인슐린으로 조정하는 방향을 고려하고 있습니다.
- 혈압약은 계속 복용 중이며, 콜레스테롤 낮추는 약도 이어서 복용합니다.

앞으로의 계획과 기대
- 메트포르민 성분의 당뇨약은 다시 사용하지 않는 것이 안전합니다.
- 인슐린 주사 방법, 용량, 주사 시간에 대해 교육을 드리고, 가급적 하루 한 번으로 단순하게 맞을 수 있도록 조정하겠습니다.
- 집에서도 혈당을 자주 확인해 주세요. 외래에서 혈당과 콩팥 기능을 정기적으로 추적 검사하겠습니다.
- 수분을 충분히 드시고, 설사·식욕저하가 지속되거나 극심한 피로, 숨이 가쁨, 복통, 어지러움/실신 느낌이 있으면 바로 병원에 연락해 주세요.
- 혈압이 너무 낮게 느껴지면(심한 어지러움, 쓰러질 것 같음 등) 알려주세요. 필요 시 혈압약을 조정하겠습니다.
- 식사 조절과 가벼운 운동을 병행하면 혈당·혈압·콜레스테롤 관리에 도움이 됩니다.

전반적으로, 약을 중단하고 수액 치료 후 상태가 호전되었고, 콩팥도 안

In [28]:
prompt_doctor = f"""
Task: Summarize the following clinical note for a clinician.
- Use concise, professional clinical language.
- Include key diagnostic findings, relevant history, differential considerations, treatments given, response to treatment, and recommended next steps.
- Maintain clinical terminology and abbreviations (e.g., BP, HR, WNL, SOB, ASA, MRI).
- Do not simplify medical terms.
- Keep the summary focused and structured.

Output language: Korean (clinician-level medical terminology)

Clinical Note:
{note}
"""

In [30]:
## Clinician Summary
model = "gpt-5" # select model

response = client.responses.create(
    model=model,
    instructions="You are a clinical text summarization assistant.",
    input=prompt_doctor
)

print(response.output_text)

임상요약

진단/배경
- T2DM (2010 진단), 최근 HbA1c 9.1%로 혈당조절 불량.
- HTN.
- CKD stage 3 (baseline Cr 1.6 mg/dL).
- Dyslipidemia.

주요 소견
- Metformin 복용 후 설사, 복부팽만, 식욕저하, 무기력감 발생.
- 입원 시 Lac 3.9 mmol/L로 lactic acidosis 의심, BP 92/58 mmHg.
- 입원 시 Cr 2.1 mg/dL로 AKI on CKD 소견. 수액 이후 신기능 호전, 투석 불필요.
- 인슐린(Levemir) 저녁 투여 순응도 저조. 기존 22 U 사용 이력 있으나 최근 10 U 수준으로 감소 추정.

치료 및 경과
- Metformin 중단, 수액치료 시행 → 증상 호전 및 대사성 이상 회복(MTF-induced lactic acidosis 의심).
- Basal insulin 전환/강화 계획. 저녁 인슐린 비순응 있음.
- Losartan 유지(저혈압 상태에서 주의 깊게 관찰).
- Atorvastatin 유지.

해석/감별
- Lactic acidosis: CKD stage 3 배경에서의 MTF-associated lactic acidosis(MALA) 가능성 높음. GI 증상 및 MTF 중단 후 호전으로 약제 연관성 지지.
- AKI on CKD는 저혈압/탈수 가능성 및 MTF 관련 대사 이상 동반 상황에서 악화된 것으로 추정. 수액 후 가역적 호전.

권고/다음 단계
- 당뇨병 관리:
  - Metformin 재투여 금지(과거 MALA 의심 + CKD3).
  - Basal insulin 시작/조정: 예) detemir 10 U QHS 시작 후 FBG 목표 80–130 mg/dL 기준 2–3일마다 2 U씩 증량(저혈당 시 감량). 저녁 순응도 문제 시 1회 투여가 용이한 장시간형(glargine U300 또는 degludec) 고려.
  - CKD3 및 ASCVD/신장보호 관점에서 SGLT2i(eGFR 허용 시) 또는 GLP-1 RA 병

# Extracting InBody report images into structured Excel data

이 Task에서는 **비정형 데이터인 Inbody Report 이미지**를 분석하여, 필요한 측정값을 **구조화된 Excel 데이터 형태**로 **자동 정리**하는 작업을 수행합니다.
- 인바디 리포트는 체성분(체중, 체지방률, 골격근량 등), 기초대사량, 신체 균형 지표 등 다양한 정보를 이미지 형태로 포함하고 있어, 직접 수기로 정리하기 어렵고 시간이 많이 소요됩니다.
- 본 튜토리얼에서는 GPT 모델을 활용하여 인바디 이미지에서 **핵심 수치·측정값을 자동으로 추출**하고, 이를 엑셀(Excel) 형태의 구조화된 표 (tabular data)로 변환하는 방법을 다룹니다.

In [35]:
## Loaded image
image_path = f"{PATH}inbody_sample.jpg"

## encoding image
with open(image_path, "rb") as f:
    img_bytes = f.read()
    img_base64 = base64.b64encode(img_bytes).decode("utf-8")

In [40]:
prompt_inbody = f"""
Your task:
- Carefully read the InBody report image.
- Extract ONLY the required values listed in the template below.
- Extract all measurement values together with their units (e.g., kg, %, kcal) exactly as shown in the image.
- For "Total Body Water", "Protein", "Minerals", and "Body Fat Mass", extract the number shown under the "Values" column of the table.
- For "Soft Lean Mass", "Fat Free Mass", and "Weight", extract the main numeric value shown in the middle of each bar graph section (do NOT use the normal range markers).
- For "Segmental Lean Analysis", each body part shows two numeric values in kg; extract ONLY the first (top) value, which represents the actual measured lean mass.
- Do NOT extract the second value (ideal/standard comparison).
- Output MUST strictly follow the English template structure provided.
- If a value is not visible, leave it blank ("").
- Do NOT hallucinate or infer values.
- Use only numbers and units that appear in the image.

Output Format:
## Basic Information
1. ID:
2. Height:
3. Age:
4. Gender:
5. Test Date/Time:

## Body Composition Analysis
1. Total Body Water Values:
2. Protein Values:
3. Minerals Values:
4. Body Fat Mass Values:
5. Soft Lean Mass:
6. Fat Free Mass:
7. Weight:

## Muscle-Fat Analysis
1. Weight:
2. Skeletal Muscle Mass (SMM):
3. Body Fat Mass:

## Obesity Analysis
1. BMI:
2. Percent Body Fat (PBF):

## Segmental Lean Analysis (kg)
1. Right Arm:
2. Left Arm:
3. Trunk:
4. Right Leg:
5. Left Leg:

Return ONLY the above template filled with extracted values.
Do not add comments, explanations, or interpretation.
"""

In [41]:
response = client.responses.create(
    model="gpt-5",
    instructions="You are an AI assistant that extracts structured measurement values from an InBody body composition analysis report image.",
    input=[
        {
            "role": "user",
            "content": [
                {"type": "input_text", "text": prompt_inbody},
                {
                    "type": "input_image",
                    "image_url": f"data:image/jpeg;base64,{img_base64}"
                }
            ]
        }
    ]
)

print(response.output_text)

## Basic Information
1. ID: Jane Doe
2. Height: 163 cm
3. Age: 41
4. Gender: Female
5. Test Date/Time: 2017.03.08, 16:47

## Body Composition Analysis
1. Total Body Water Values: 35.5 L
2. Protein Values: 9.2 kg
3. Minerals Values: 3.28 kg
4. Body Fat Mass Values: 18.1 kg
5. Soft Lean Mass: 45.6 kg
6. Fat Free Mass: 48.3 kg
7. Weight: 66.4 kg

## Muscle-Fat Analysis
1. Weight: 66.4 kg
2. Skeletal Muscle Mass (SMM): 26.7 kg
3. Body Fat Mass: 18.1 kg

## Obesity Analysis
1. BMI: 25.0 kg/m²
2. Percent Body Fat (PBF): 27.2 %

## Segmental Lean Analysis (kg)
1. Right Arm: 2.56 kg
2. Left Arm: 2.60 kg
3. Trunk: 20.9 kg
4. Right Leg: 7.59 kg
5. Left Leg: 7.59 kg


In [43]:
def parse_inbody_text_to_row(text):
    """
    InBody 텍스트 블록을 받아서 {'항목명': 값} 딕셔너리로 파싱
    """
    row_data = {}

    lines = text.strip().split("\n") # text를 줄 단위로 나누기

    for line in lines:
        line = line.strip()

        # section header는 건너뛰기
        if line.startswith("##"):
            continue

        # key:value 패턴 매칭
        """
        (?:\d+\.\s*)? = 1. 같은 번로가 있어도 무시 (있던 없던 처리 가능)
        (.+?) = : 앞에 오는 문자열을 key로 (공백, 괄호 포함 가능)
        \s*(.*) = : 뒤에 오는 내용을 value로
        """
        match = re.match(r"(?:\d+\.\s*)?(.+?):\s*(.*)", line)

        if match:
            key = match.group(1).strip()
            value = match.group(2).strip()

            # 빈 값만 아닌 경우 저장
            row_data[key] = value

    return row_data

In [44]:
parsed_row = parse_inbody_text_to_row(response.output_text)
df = pd.DataFrame([parsed_row])
df.head()

,ID,Height,Age,Gender,Test Date/Time,Total Body Water Values,Protein Values,Minerals Values,Body Fat Mass Values,Soft Lean Mass,...,Weight,Skeletal Muscle Mass (SMM),Body Fat Mass,BMI,Percent Body Fat (PBF),Right Arm,Left Arm,Trunk,Right Leg,Left Leg
0,Jane Doe,163 cm,41,Female,"2017.03.08, 16:47",35.5 L,9.2 kg,3.28 kg,18.1 kg,45.6 kg,...,66.4 kg,26.7 kg,18.1 kg,25.0 kg/m²,27.2 %,2.56 kg,2.60 kg,20.9 kg,7.59 kg,7.59 kg


In [47]:
excel_path = f"{PATH}inbody_sample.csv"
df.to_csv(excel_path, index=False)